In [1]:
from pyspark import SparkConf, SparkContext
import os

conf = SparkConf().setAppName("CheckPythonEnv").setMaster("spark://master:7077")
sc = SparkContext(conf=conf)

# 在集群上运行命令来查看 Python 解释器路径
python_path_rdd = sc.parallelize([1]).map(lambda x: os.popen("which python").read().strip())
print("Python interpreter path on Spark nodes:", python_path_rdd.collect())

sc.stop()

Python interpreter path on Spark nodes: ['/home/ecs-user/anaconda3/envs/big_data_lab_env/bin/python']


In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
import tempfile

conf = SparkConf().setAppName("CHN").setMaster("spark://master:7077")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
spark = SparkSession(sc)

print("Spark 配置成功")

TRAIN_PATH = "/chn/train.csv"
TEST_PATH = "/chn/test.csv"

print("-----加载训练数据-----")
rdd_train = sc.textFile(TRAIN_PATH)
rdd_train = rdd_train.map(lambda line: LabeledPoint(float(line.split(",")[-1]), 
                                                    Vectors.dense([float(x) for x in line.split(",")[:-1]])))
print("-----训练数据加载成功-----")

print("-----加载测试数据-----")
rdd_test = sc.textFile(TEST_PATH)
rdd_test = rdd_test.map(lambda line: LabeledPoint(float(line.split(",")[-1]), 
                                                  Vectors.dense([float(x) for x in line.split(",")[:-1]])))

print("-----测试数据加载成功-----")

# # 打印训练数据和测试数据的前几行
# print("-----训练数据-----")
# print(rdd_train.take(5))
# print("-----测试数据-----")
# print(rdd_test.take(5))

Spark 配置成功
-----加载训练数据-----
-----训练数据加载成功-----
-----加载测试数据-----
-----测试数据加载成功-----


In [3]:
model = LogisticRegressionWithLBFGS().train(rdd_train, iterations=100, numClasses=15)

print("-----模型训练成功-----")

model_save_path = tempfile.mkdtemp()
model.save(sc, model_save_path)
print("-----模型保存成功-----")

print("-----模型评估中-----")
score_and_labels = rdd_test.map(lambda point: (model.predict(point.features), point.label))
accuracy = score_and_labels.filter(lambda l: l[0] == l[1]).count() / rdd_test.count()
print("模型准确率:", accuracy)

sc.stop()

09:33:41.496 [task-result-getter-0] WARN  org.apache.spark.scheduler.TaskSetManager - Lost task 0.0 in stage 0.0 (TID 0) (172.29.118.232 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1227, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(obj, encoding=encoding)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/mllib/__init__.py", line 26, in <module>
    import numpy
ModuleNotFoundError: No module named 'numpy'

	at org.apache.spark.api.p

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (172.29.118.232 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1227, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(obj, encoding=encoding)
  File "/usr/local/spark/python/lib/pyspark.zip/pys